In [ ]:
import torch
import math
import matplotlib.pyplot as plt

tri1 = torch.tensor([[0, 0], [1, 0], [0.5, math.sqrt(3)/2]])
tri2 = torch.tensor([[0, 0], [1, 0], [0.9, 0.6]])

plt.figure(figsize=(3, 3))
plt.gca().add_patch(plt.Polygon(tri1, edgecolor='black', fill=None))
plt.gca().add_patch(plt.Polygon(tri2, edgecolor='blue', fill=None))

plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
# compute affine matrix (non-batch)
homo_ones = torch.ones((3, 1))
E = torch.cat([tri1, homo_ones], dim=1).T
A = torch.cat([tri2, homo_ones], dim=1).T
M = A @ torch.inverse(E)

# affine matrix M: from equilateral to current triangle, ME = A 
# 1. no shift, no scale/direction change at x-axis, so
# M must be [[1, m1], [0, m2]], with m1, m2 to be solved.
# 2. assume only the third pair of points E_x/y, A_x/y change,
# m1 = (A_x-E_x)/E_y, m2 = A_y/E_y
M_2d = torch.tensor([
    [1., (tri2[2,0]-tri1[2,0])/tri1[2,1]], 
    [0., tri2[2,1]/tri1[2,1]],
])

In [ ]:
# gaussian settings on a triangle
barycentric_coords =  [[2/3, 1/6, 1/6], [1/6, 2/3, 1/6], [1/6, 1/6, 2/3],
                       [1/6, 5/12, 5/12], [5/12, 1/6, 5/12], [5/12, 5/12, 1/6]]
barycentric_coords =  [[(3-(3**0.5))/6, (3-(3**0.5))/6, (3**0.5)/3], 
                       [(3-(3**0.5))/6, (3**0.5)/3, (3-(3**0.5))/6], 
                       [(3**0.5)/3, (3-(3**0.5))/6, (3-(3**0.5))/6]]

def barycentric_to_cartesian(triangle, barycentric_coords):
    return sum(vertex * weight for vertex, weight in zip(triangle, barycentric_coords))


# Group-1: Gaussians on equilateral triangle
mean_1_list = [barycentric_to_cartesian(tri1, coords) for coords in barycentric_coords]
s = 1 / (4. + 2.*(3**0.5))  # we assume scale=sqrt(eigenvalue)
s = 1 / (2. + 2.*(3**0.5))  # we assume scale=sqrt(eigenvalue)
conv_1 = torch.diag(torch.tensor([s**2, s**2]))

In [ ]:
# Group-2: Gaussians on transformed triangle
mean_2_list = [barycentric_to_cartesian(tri2, coords) for coords in barycentric_coords]

# Method-1: do affine on conv matrix
conv_2 = M_2d @ (conv_1 @ M_2d.T)

# # Method-2: do affine on S
# U, S_values, Vt = torch.linalg.svd(M_2d)
# s_ = torch.diag(S_values * torch.tensor([s, s]))
# conv_2 = U @ s_ @ s_.T @ Vt

In [ ]:
from matplotlib.patches import Ellipse

def gaussian_to_ellipse(cov_matrix):
    eigenvalues, eigenvectors = torch.linalg.eigh(cov_matrix)
    return torch.sqrt(eigenvalues), eigenvectors

def create_ellipse(mean, std_devs, rotation, ax, color):
    angle = torch.rad2deg(torch.atan2(rotation[1, 0], rotation[0, 0]))
    ellipse = Ellipse(xy=mean.numpy(), width=2*std_devs[0].item(), height=2*std_devs[1].item(), angle=angle.item(), edgecolor=color, fc='None', lw=2)
    ax.add_patch(ellipse)

fig, ax = plt.subplots()

std_dev1, rotation1 = gaussian_to_ellipse(conv_1)
for mean_1 in mean_1_list:
    create_ellipse(mean_1, std_dev1, rotation1, ax, 'black')

std_dev2, rotation2 = gaussian_to_ellipse(conv_2)
for mean_2 in mean_2_list:
    create_ellipse(mean_2, std_dev2, rotation2, ax, 'blue')

plt.gca().add_patch(plt.Polygon(tri1, edgecolor='black', fill=None))
plt.gca().add_patch(plt.Polygon(tri2, edgecolor='blue', fill=None))

plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
plt.gca().set_aspect('equal', adjustable='box')
plt.grid(True)
plt.show()
